In [25]:
from pprint import pprint

import requests
import yaml

import argo_workflows
from argo_workflows.api import workflow_template_service_api

configuration = argo_workflows.Configuration(host="https://127.0.0.1:2746")
configuration.verify_ssl = False

# Workflow template objects

We have a closer look at the data structure of the `WorkflowTemplate` CRD as returned by the `argo` SDKs `workflow_template_service_api`s `WorkflowServiceApi.list_workflows` / `WorkflowServiceApi.get_workflow` methods/APIs.

Understanding these structures will helps us build the required meta and status data retrievers for the frontend, and also during the devleopment of the `bettmensch.ai.pipelines` SDK.

In [31]:
api_client = argo_workflows.ApiClient(configuration)
api_instance = workflow_template_service_api.WorkflowTemplateServiceApi(api_client)

# get all workflows
workflow_templates = api_instance.list_workflow_templates(namespace="argo")
[workflow_template['metadata']['name'] for workflow_template in workflow_templates['items']]

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['set-a-coin-rl4sj']

In [36]:
first_workflow_template = api_instance.get_workflow_template(namespace="argo",name="set-a-coin-rl4sj").to_dict()

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [39]:
first_workflow_template.keys() # > dict_keys(['metadata', 'spec']).

dict_keys(['metadata', 'spec'])

## `.metadata`

The `WorkflowTemplate` manifest's `metadata` field. Holds identifiers and labels.

In [68]:
first_workflow_template['metadata'].keys() # > dict_keys(['name', 'generate_name', 'namespace', 'uid', 'resource_version', 'generation', 'creation_timestamp', 'labels', 'managed_fields'])
(
    first_workflow_template['metadata']['name'], 
    first_workflow_template['metadata']['generate_name'], 
    first_workflow_template['metadata']['namespace'], 
    first_workflow_template['metadata']['uid'], 
    first_workflow_template['metadata']['resource_version'], 
    first_workflow_template['metadata']['generation'], 
    first_workflow_template['metadata']['creation_timestamp'], 
    first_workflow_template['metadata']['labels'],
    first_workflow_template['metadata']['managed_fields']
)
# > ('set-a-coin-rl4sj',
#  'set-a-coin-',
#  'argo',
#  '59c2a783-2ed7-4ad6-b852-ad94d45dc09c',
#  '63522',
#  1,
#  datetime.datetime(2024, 5, 1, 11, 12, 13, tzinfo=tzutc()),
#  {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'},
#  [{'manager': 'argo',
#    'operation': 'Update',
#    'api_version': 'argoproj.io/v1alpha1',
#    'time': datetime.datetime(2024, 5, 1, 11, 12, 13, tzinfo=tzutc()),
#    'fields_type': 'FieldsV1',
#    'fields_v1': {'f:metadata': {'f:generateName': {},
#      'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}},
#     'f:spec': {}}}])

('set-a-coin-rl4sj',
 'set-a-coin-',
 'argo',
 '59c2a783-2ed7-4ad6-b852-ad94d45dc09c',
 '63522',
 1,
 datetime.datetime(2024, 5, 1, 11, 12, 13, tzinfo=tzutc()),
 {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'},
 [{'manager': 'argo',
   'operation': 'Update',
   'api_version': 'argoproj.io/v1alpha1',
   'time': datetime.datetime(2024, 5, 1, 11, 12, 13, tzinfo=tzutc()),
   'fields_type': 'FieldsV1',
   'fields_v1': {'f:metadata': {'f:generateName': {},
     'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}},
    'f:spec': {}}}])

## `.spec`

The `WorkflowTemplate`'s manifest's `spec` section. This is where the overall DAG is defined via 
- the `templates` field, containing the `task` definitions as modular function definition, including the `entrypoint` template
- the `entrypoint` field, specifying the template that serves as the DAG's head node
- the `arguments` field that lists all the `WorkflowTemplate` level input arguments, with *optional* default values. In our case, these will be exclusively of type `Paramter`.

As we have specified the `pod_metadata` and `workflow_metadata` fields in our example, we will find those optional sections populated here as well. They will eventually be merged into the downstream `Workflow` resource manifest at the `Workflow` and `Pod`/`Node` level, respectively. 

Will be inherited by the `Workflow` - see also See `5_argo_workflow_objects.ipynb`'s `status.store_workflow_template_spec` section.

In [48]:
# spec is the Workflow manifest `spec` manifest field and so contains the 
first_workflow_template['spec'].keys() # >dict_keys(['templates', 'entrypoint', 'arguments', 'pod_metadata', 'workflow_metadata'])

dict_keys(['templates', 'entrypoint', 'arguments', 'pod_metadata', 'workflow_metadata'])

### `.spec.arguments`

In [50]:
first_workflow_template['spec']['arguments'] # specified arguments (defined by the WorkflowTemplate) with values (specified in the Workflow construction). For us, these will only be `Parameter` type input arguments.
# > {'parameters': [{'name': 'coin'}]}

{'parameters': [{'name': 'coin'}]}

### `.spec.entrypoint`

In [52]:
first_workflow_template['spec']['entrypoint'] # the `name` of the template defined as part of the `WorkflowTemplate` that serves as the entrypoint to the entire DAG
# > 'Coin-set'

'Coin-set'

### `spec.templates`

Lists all the `template` definitions. 

The most interesting here is the template that is used as the entrypoint for the `WorkflowTemplate`: It defines the DAG structure in its `dag` field. 

Each task listed there refers to another template, and contains (where applicable) a reference to the `task`'s/DAG node's parent node. using this information, we can recursively build up the graph structure of the `WorkflowTemplate`'s DAG.

In [59]:
first_workflow_template['spec']['templates'] # the list of defined templates in the WorkflowTemplate manifest, including the one serving as entrypoint. Notice the additional `dag` field in the task named "Coin-set" (the entrypoint task).

[{'name': 'Coin-set',
  'inputs': {},
  'outputs': {},
  'metadata': {},
  'dag': {'tasks': [{'name': 'Set-a-coin',
     'template': 'set-coin',
     'arguments': {'parameters': [{'name': 'coin',
        'value': '{{workflow.parameters.coin}}'}]}},
    {'name': 'Show-a-coin',
     'template': 'show-coin',
     'arguments': {'parameters': [{'name': 'coin',
        'value': '{{tasks.Set-a-coin.outputs.parameters.coin}}'}]},
     'depends': 'Set-a-coin'}]}},
 {'name': 'set-coin',
  'inputs': {'parameters': [{'name': 'coin'}]},
  'outputs': {'parameters': [{'name': 'coin',
     'value_from': {'path': './coin_output.txt'}}]},
  'metadata': {},
  'script': {'image': 'python:3.8',
   'source': "import os\nimport sys\nsys.path.append(os.getcwd())\nimport json\ntry: coin = json.loads(r'''{{inputs.parameters.coin}}''')\nexcept: coin = r'''{{inputs.parameters.coin}}'''\n\nwith open('./coin_output.txt', 'w') as output:\n    output.write(coin)",
   'name': '',
   'command': ['python'],
   'resource

In [61]:
first_workflow_template['spec']['templates'][0] # entrypoint template "Coin set" - compare to `spec.entrypoint` entry. Notice its `dag` field defining the DAG structure

{'name': 'Coin-set',
 'inputs': {},
 'outputs': {},
 'metadata': {},
 'dag': {'tasks': [{'name': 'Set-a-coin',
    'template': 'set-coin',
    'arguments': {'parameters': [{'name': 'coin',
       'value': '{{workflow.parameters.coin}}'}]}},
   {'name': 'Show-a-coin',
    'template': 'show-coin',
    'arguments': {'parameters': [{'name': 'coin',
       'value': '{{tasks.Set-a-coin.outputs.parameters.coin}}'}]},
    'depends': 'Set-a-coin'}]}}

In [62]:
first_workflow_template['spec']['templates'][1]

{'name': 'set-coin',
 'inputs': {'parameters': [{'name': 'coin'}]},
 'outputs': {'parameters': [{'name': 'coin',
    'value_from': {'path': './coin_output.txt'}}]},
 'metadata': {},
 'script': {'image': 'python:3.8',
  'source': "import os\nimport sys\nsys.path.append(os.getcwd())\nimport json\ntry: coin = json.loads(r'''{{inputs.parameters.coin}}''')\nexcept: coin = r'''{{inputs.parameters.coin}}'''\n\nwith open('./coin_output.txt', 'w') as output:\n    output.write(coin)",
  'name': '',
  'command': ['python'],
  'resources': {}}}

In [65]:
first_workflow_template['spec']['templates'][2] # this template/DAG node depends on the template "set-coin", but this dependency is a property of the DAG (the "Coin-set" template) and not of the DAG node (this "show-coin" template)

{'name': 'show-coin',
 'inputs': {'parameters': [{'name': 'coin'}]},
 'outputs': {},
 'metadata': {},
 'script': {'image': 'python:3.8',
  'source': "import os\nimport sys\nsys.path.append(os.getcwd())\nimport json\ntry: coin = json.loads(r'''{{inputs.parameters.coin}}''')\nexcept: coin = r'''{{inputs.parameters.coin}}'''\n\nprint(f'it was {coin}')",
  'name': '',
  'command': ['python'],
  'resources': {}}}